In [1]:
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from keras.models import Sequential, load_model, Model,Input
from keras.layers import Dense,GlobalAvgPool2D, Conv2D, MaxPool2D,AvgPool2D, GlobalAveragePooling2D,AveragePooling2D ,Softmax, BatchNormalization, ReLU, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
from keras.layers import concatenate,Concatenate
import matplotlib.pyplot as plt
from matplotlib.image import imread 
%matplotlib inline
import pandas as pd
import numpy as np
import cv

In [2]:
# preprocessing the training set
train_data_generation = ImageDataGenerator(rescale=1./255)
training_set = train_data_generation.flow_from_directory('/home/fatmaosama/Desktop/GP/PartA_Splitted/train',
                                                         target_size=(224, 224),
                                                         batch_size=32,
                                                         class_mode='binary',seed=0)

# preprocessing the testing set
test_data_generation = ImageDataGenerator(rescale=1./255)
testing_set = test_data_generation.flow_from_directory('/home/fatmaosama/Desktop/GP/PartA_Splitted/val',
                                                       target_size=(224, 224),
                                                       batch_size=32,
                                                       class_mode='binary',seed=0)

Found 1174 images belonging to 2 classes.
Found 505 images belonging to 2 classes.


In [3]:
def densenet(img_shape, f=32):
  repetitions = [6, 12, 24, 16]
  
  def bn_rl_conv(model, f, k=1, s=1, p='same'):
    model = BatchNormalization()(model)
    model = ReLU()(model)
    model = Conv2D(f, k, strides=s, padding=p)(model)
    return model
  
  
  def dense_block(conc, r):
    for i in range(r):
      model = bn_rl_conv(conc, 4*f)
      model = bn_rl_conv(model, f, 3)
      conc = Concatenate()([conc, model])
    return conc
  
  
  def transition_block(model):
    model = bn_rl_conv(model, K.int_shape(model)[-1] // 2)
    model = AvgPool2D(2, strides=2, padding='same')(model)
    return model
  
  
  input = Input(img_shape)
  
  model = Conv2D(64, 7, strides=2, padding='same')(input)
  model = MaxPool2D(3, strides=2, padding='same')(model)
  
  for r in repetitions:
    d = dense_block(model, r)
    model = transition_block(d)
  
  model = GlobalAvgPool2D()(d)
  
  output = Dense(2, activation='softmax')(model)
  
  model = Model(input, output)
  return model

In [4]:
model= densenet([224,224,3])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 56, 56, 64)   0           ['conv2d[0][0]']                 
                                                                                              

In [9]:
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x=training_set, validation_data=testing_set, epochs=50)


Epoch 1/50
37/37 [==============================] - 297s 8s/step - loss: 0.4738 - accuracy: 0.7888 - val_loss: 218.8085 - val_accuracy: 0.6178
Epoch 2/50
37/37 [==============================] - 288s 8s/step - loss: 0.3835 - accuracy: 0.8279 - val_loss: 118.9808 - val_accuracy: 0.6178
Epoch 3/50
37/37 [==============================] - 288s 8s/step - loss: 0.3600 - accuracy: 0.8424 - val_loss: 1.2267 - val_accuracy: 0.7465
Epoch 4/50
37/37 [==============================] - 287s 8s/step - loss: 0.3312 - accuracy: 0.8578 - val_loss: 0.7938 - val_accuracy: 0.7307
Epoch 5/50
37/37 [==============================] - 287s 8s/step - loss: 0.3776 - accuracy: 0.8492 - val_loss: 7.6094 - val_accuracy: 0.3960
Epoch 6/50
37/37 [==============================] - 287s 8s/step - loss: 0.3370 - accuracy: 0.8612 - val_loss: 1.3965 - val_accuracy: 0.6693
Epoch 7/50
37/37 [==============================] - 287s 8s/step - loss: 0.3179 - accuracy: 0.8773 - val_loss: 1.0392 - val_accuracy: 0.7644
Epoch 8/5

In [6]:
#Preprocessing
train_data_generation1 = ImageDataGenerator(rescale=1./255,
                                           shear_range=0.2,
                                           zoom_range=0.2,
                                           horizontal_flip=True)

#preprocessing the training set                                          
training_set1 = train_data_generation1.flow_from_directory('/home/fatmaosama/Desktop/GP/PartA_Splitted/train',
                                                         target_size=(224, 224),
                                                         batch_size=32,
                                                         class_mode='binary'
                                                         ,seed=0)

#preprocessing the testing set
test_data_generation1 = ImageDataGenerator(rescale=1./255)
testing_set1 = test_data_generation1.flow_from_directory('/home/fatmaosama/Desktop/GP/PartA_Splitted/val',
                                                       target_size=(224, 224),
                                                       batch_size=32,
                                                       class_mode='binary',seed=0)



Found 1174 images belonging to 2 classes.
Found 505 images belonging to 2 classes.


In [8]:
model1= densenet([224,224,3])
model1.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model1.fit(x=training_set1, validation_data=testing_set1, epochs=50)

Epoch 1/50
37/37 [==============================] - 272s 7s/step - loss: 0.9874 - accuracy: 0.6755 - val_loss: 4.0810 - val_accuracy: 0.2911
Epoch 2/50
37/37 [==============================] - 279s 8s/step - loss: 0.5135 - accuracy: 0.7351 - val_loss: 0.7578 - val_accuracy: 0.4198
Epoch 3/50
37/37 [==============================] - 294s 8s/step - loss: 0.4524 - accuracy: 0.7913 - val_loss: 1.1503 - val_accuracy: 0.3822
Epoch 4/50
37/37 [==============================] - 300s 8s/step - loss: 0.4509 - accuracy: 0.7811 - val_loss: 0.9791 - val_accuracy: 0.3941
Epoch 5/50
37/37 [==============================] - 300s 8s/step - loss: 0.4235 - accuracy: 0.7939 - val_loss: 1.3403 - val_accuracy: 0.3881
Epoch 6/50
37/37 [==============================] - 300s 8s/step - loss: 0.4309 - accuracy: 0.7871 - val_loss: 0.7334 - val_accuracy: 0.4911
Epoch 7/50
37/37 [==============================] - 303s 8s/step - loss: 0.4612 - accuracy: 0.7964 - val_loss: 0.7613 - val_accuracy: 0.5010
Epoch 8/50
37